In [1]:
import re
import spacy
import pandas as pd 
from unidecode import unidecode
from collections import Counter
import spacy
from spaczz.matcher import FuzzyMatcher
nlp = spacy.blank("es")

# Preprocesamiento de texto

In [2]:
#Funcion inicial de limpieza de texto 
def Limpiar_texto(texto):
    texto = unidecode(texto)
    # Eliminar cualquier signo de puntuación
    puntuación = r'[.,;:¡!¿?@#$%&[\](){}<>~=+\*/|\\_^`"\']'
    #puntuación = r'[^\w\s]|(?<=\w)\.(?=[a-zA-Z])'
    texto = re.sub(puntuación, '', texto)

    # Eliminar cualquier carácter que no sea una letra o un número
    alfanumérico = r'\w+(?:-\w+)*'
    texto = re.sub(alfanumérico, lambda x: x.group(0).replace('-', ' '), texto)

    # Eliminar cualquier caracter que no sea una letra o un número o un espacio en blanco
    texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto)
    # eliminar espacios en blanco iniciales
    texto = re.sub(r"^\s+", "", texto)
    doc = nlp(texto.lower())

    return " ".join(unidecode(token.norm_) for token in doc if not token.is_punct)


In [3]:
#Se lee el documento
accionados = pd.read_excel('accionado.xlsx')
#se aplica la funcion "limpiar_texto" sobre la columna que tiene los accionados y se crea una nueva columna con el texto limpio
accionados['texto_limpio'] = accionados['Demandado'].apply(Limpiar_texto)    

# Etiqueta_Otros

In [4]:
#se borra del texto limpio la palabra "otros" y se crea una columna llamada "otros" con un etiquetado binario
otros = r'y\s?otro(s)?'
accionados = accionados.assign(otros=accionados['Demandado'].astype(str).str.contains(otros, flags=re.IGNORECASE).astype(int))
accionados['texto_limpio'] = accionados['texto_limpio'].str.replace(otros, ' ', regex=True)

C:\Users\Equipo\AppData\Local\Temp\ipykernel_14604\1737366966.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  accionados = accionados.assign(otros=accionados['Demandado'].astype(str).str.contains(otros, flags=re.IGNORECASE).astype(int))


# Stop_words

In [5]:
#listado inicial de stop words que se eliminan  
stop_words = ["sa", "sas", "ltda", "y", "e", "ni", "que", "pero", "si", "porque", "cuando", "pues", "sino",
            "aunque","el","los","las","lo","la", "un","una","unos","unas","al","del",
            "a","ante","bajo","cabe","con","contra","de","desde",
            "en","entre","hacia","hasta","para",
            "por" ,"segun" ,"sin" ,"so" ,"sobre" ,"tras"]
#se eliminan los espacios dobles 
accionados['texto_limpio'] = accionados['texto_limpio'].apply(lambda x: re.sub(r'(\b\w\s){2,}\w\b', lambda x: x.group().replace(' ', ''), x))
#se remplazan las stop words por un espacio vacio
patron = r"\b(" + "|".join(stop_words) + r")\b"
accionados['texto_limpio'] = accionados['texto_limpio'].apply(lambda x: re.sub(patron,"",x))
#se eliminan los excesos de espacios
accionados['texto_limpio'] = accionados['texto_limpio'].apply(lambda x: re.sub(' +', ' ', x))
#se elimina espacios en blanco iniciales
accionados['texto_limpio'] = accionados['texto_limpio'].apply(lambda x: re.sub(r"^\s+", "",x))

# Estandarizado expresiones regulares

In [6]:
#El listado de departamentos se utiliza para detectar cual gobernacion corresponde 
departamentos = ["amazonas", "antioquia", "arauca", "atlantico", "bolivar", "boyaca", "caldas", "caqueta", "casanare", "cauca", "cesar", "choco", "cordoba", "cundinamarca", "guainia", "guaviare", "huila", "guajira", "magdalena", "meta", "narino", "norte santander", "putumayo", "quindio", "risaralda", "san andres providencia", "santander", "sucre", "tolima", "valle", "valle cauca", "vaupes", "vichada"]

#Esta biblioteca contiene las regex de las entidades
entidades = {
        "uariv": r"^(?!.*\bunp\b)(?=.*unidad)(?=.*victimas).*|(uariv).*$",
        #alcaldias
        "alcaldia": r"alcaldia(s)?",
        "gobernacion": r"gobernacion",
        #Pensiones
        "colpensiones": r"colpension(es)?|^(?=.*administradora)(?=.*colombiana)(?=.*pensiones).*",
        "colfondos": r"colfondo(s)?",
        "skandia afp": r"skandia",
        "proteccion afp": r"^(?!.*(ministerio|contribucion(es)?|alfa|nacional|social|policia|unp|inspeccion|eps|unidad|seguridad|grupo|secretaria|empleo|empleados)).*proteccion|(?=.*proteccion)(?=.*afp).*$",
        "porvenir afp": r"^(?!.*(alcaldia|clinica|condominio|sendero(s)?|secretaria|inspeccion|comunal|conjunto|junta|centro|comercial|asociacion|consejo|inspeccion|residencial|condado(s)?)).*porvenir|(?=.*porvenir)(?=.*afp).*$",
        
        #Eps
        "asmet salud eps": r"asme(.*)[dt]",
        "asociacion indigena cauca aic eps": r"\baic\b|^(?=.*asociacion)(?=.*indigena)(?=.*cauca).*",
        "aliansalud eps": r"(alian\s?salud|alianza salud)",
        "ambuq eps": r"ambuq|^(?=.*asociacion)(?=.*mutual)(?=.*barrios)(?=.*unidos)(?=.*quibdo).*",
        "anas wayuu eps": r"^(?=.*anas)(?=.*wayu(u)?).*",
        "cajacopi eps": r"^(?=.*caja\s?copi)(?=.*eps)",
        "capital salud eps": r"(?=.*capital)(?=.*salud)|(?=.*capital)(?=.*eps)",
        "capresoca eps": r"capresoca",
        "coomeva eps": r"(?=.*co(o)?meva)(?=.*eps)",
        "coosalud eps": r"co(o)?salud",
        "comparta eps": r"(?=.*comparta)(?=.*eps)",
        "compensar eps": r"(?=.*compensar)(?=.*eps)",
        "comfachoco eps": r"(?=.*comfa\s?choco)(?=.*eps)",
        "comfaca eps": r"(?=.*comfa\s?ca)(?=.*eps)",
        "comfacor eps": r"(?=.*comfa\s?cor)(?=.*eps)",
        "comfasucre eps": r"(?=.*comfa\s?sucre)(?=.*eps)",
        "comfanorte eps": r"(?=.*comfa\s?norte)(?=.*eps)",
        "comfaoriente eps": r"(?=.*comfa\s?oriente)(?=.*eps)",
        "comfamiliar eps": r"(?=.*comfamiliar)(?=.*eps)",
        "comfenalco eps": r"(?=.*comfenalco)(?=.*eps)",
        "comfandi eps": r"(?=.*comfandi)(?=.*eps)",
        "convida eps": r"convida", 
        "cosmitet eps": r"cosmitet",
        "dusakawi eps": r"dusakawi",
        "ecoopsos eps": r"eco(o)?psos",
        "emssanar eps": r"ems(s)?anar",
        "emcosalud eps": r"(?=.*emco\s?salud)(?=.*eps)",
        "famisanar eps": r"famisanar",
        "familiar de colombia eps": r"(?=.*familiar)(?=.*colombia)(?=.*eps)",
        "mallamas eps": r"(?=.*mal(l)?a\s?ma(s)?)(?=.*eps)",
        "medimas eps": r"medimas",
        "mutualser eps": r"mutual\s?ser",
        "nueva eps": r"(?=.*nueva)(?=.*eps)|(?=.*empresa)(?=.*salud)(?=.*nueva)",
        "pijaos salud eps": r"(?=.*pijao(s)?)(?=.*salud)",
        "saludcoop eps": r"saludco(o)?p",
        "salud bolivar eps": r"(?=.*salud\s?bolivar)(?=.*eps)",
        "salud mia eps": r"salud\s?mia",
        "salud total eps": r"salud\s?total",
        "sanitas eps": r"\bsanita(s)?\b|(?=.*colsanita(s)?)(?=.*eps)",
        "savia salud eps": r"(?=.*alianza)(?=.*medellin)(?=.*antioquia)|savia|(?=.*savia)(?=.*eps)",
        "servicio occidental de salud eps": r"(?=.*\bsos\b)(?=.*eps(i)?)|(?=.*servicio)(?=.*occidental)(?=.*salud)",
        "sura eps": r"(?=.*sura)(?=.*eps)|(?=.*entidad)(?=.*salud)(?=.*sura)",
    
        #ARL
        "axa colpatria arl": r"(?=.*colpatria)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*colpatria)|(?=.*colpatria)(?=.*axa)",
        "colmena arl": r"(?=.*colmena)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*colmena)",
        "equidad seguros arl": r"(?=.*equidad)(?=.*arl)(?=.*seguro(s)?)|(?=.*administradora)(?=.*laborales)(?=.*equidad)(?=.*seguro(s)?)",
        "seguros alfa arl": r"(?=.*alfa)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*alfa)",
        "arl positiva": r"(?=.*positiva)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*positiva)",
        "arl seguros bolivar": r"(?=.*bolivar)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*bolivar)",
        "sura arl": r"(?=.*sura)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*sura)",
        "arl aurora seguros de vida":  r"(?=.*aurora)(?=.*arl)|(?=.*administradora)(?=.*laborales)(?=.*aurora)",   
    
        #ministerios 
        "ministerio del interior": r"(?=.*ministerio)(?=.*interior)|min\s?interior",
        "ministerio de relaciones exteriores": r"(?=.*ministerio)(?=.*relaciones)|cancilleria",
        "ministerio de hacienda y credito publico": r"(?=.*ministerio)(?=.*hacienda)|min\s?hacienda", 
        "ministerio de justicia y del derecho": r"(?=.*ministerio)(?=.*justicia)|min\s?justicia",
        "ministerio de defensa nacional": r"(?=.*ministerio)(?=.*defensa)|min\s?defensa",  
        "ministerio de agricultura y desarrollo rural": r"(?=.*ministerio)(?=.*agricultura)|min\s?agricultura",  
        "ministerio de salud y proteccion social": r"(?=.*ministerio)(?=.*salud)|min\s?salud",
        "ministerio de trabajo": r"(?=.*ministerio)(?=.*trabajo)|min\s?trabajo",
        "ministerio de minas y energia":  r"(?=.*ministerio)(?=.*minas)|min\s?energia",
        "ministerio de comercio industria y turismo": r"(?=.*ministerio)(?=.*comercio)|(?=.*ministerio)(?=.*industria)|(?=.*ministerio)(?=.*turismo)|min\s?cit|min\s?comercio",
        "ministerio de educacion nacional" : r"(?=.*ministerio)(?=.*educacion)|min\s?educacion",
        "ministerio de ambiente y desarrollo sostenible": r"(?=.*ministerio)(?=.*ambiente)|min\s?ambiente",
        "ministerio de vivienda ciudad y territorio": r"(?=.*ministerio)(?=.*vivienda)|min\s?vivienda",
        "ministerio de tecnologias de la informacion y las comunicaciones": r"(?=.*ministerio)(?=.*tecnologia(s)?)|min\s?tic|(?=.*ministerio)(?=.*\btic\b)",
        "ministerio de transporte":  r"(?=.*ministerio)(?=.*transporte)|min\s?transporte",
        "ministerio de cultura":  r"(?=.*ministerio)(?=.*\bcultura\b)|min\s?cultura",
        "ministerio de ciencia tecnologia e innovacion": r"(?=.*ministerio)(?=.*ciencia)|min\s?ciencia",
        "ministerio de deporte": r"(?=.*ministerio)(?=.*deporte)|min\s?deporte",
        "ministerio de igualdad": r"(?=.*ministerio)(?=.*igualdad)|min\s?igualdad|min\s?equidad",
    
        #Bancos
        'bancolombia': r"bancolombia", 
        'banco de bogota': r"(?=.*banco)(?=.*bogota)", 
        'banco davivienda': r"davivienda", 
        'banco bbva': r"bbva|(?=.*banco)(?=.*bilbao)(?=.*argentina)", 
        'banco de occidente': r"(?=.*banco)(?=.*occidente)", 
        'banco scotiabank colpatria': r"(?=.*banco)(?=.*colpatria)|scotiabank", 
        'banco itau': r"itau", 
        'banco gnb sudameris': r"(?=.*banco)(?=.*gnb)|(?=.*gnb)(?=.*sudameris)", 
        'banco agrario': r"(?=.*banco)(?=.*agrario)", 
        'banco popular': r"(?=.*banco)(?=.*popular)", 
        'banco caja social': r"banco\s+caja\s+social|bcsc", 
        'banco av villas': r"(?=.*av)(?=.*villas)", 
        'banco union': r"(?=.*banco)(?=.*union)", 
        'banco bancoomeva': r"banco(o)?meva", 
        'banco falabella': r"(?=.*banco)(?=.*falabella)", 
        'banco pichincha': r"pichincha", 
        'banco w': r"banco\s?w", 
        'banco finandina': r"finandina", 
        'banco serfinanza': r"serfinanza",
        'banco bancamia': r"bancamia", 
        'banco credifinanciera': r"credifinanciera", 
        'banco cooperativo coopcentral':  r"(?=.*banco)(?=.*co(o)?perativo)|(?=.*co(o)?pcentral)",
        'banco bancoldex': r"bancoldex",
        'banco nequi':r"nequi",
        'banco de la republica': r"(?=.*banco)(?=.*republica)",
        
        
        #Secretarias_departamentales
        "secretaria educacion":  r"(?=.*secretaria)(?=.*educacion)",
        "secretaria ambiente": r"(?=.*secretaria)(?=.*ambiente)",
        "secretaria de cultura": r"(?=.*secretaria)(?=.*cultura)",
        "secretaria hacienda": r"(?=.*secretaria)(?=.*hacienda)",
        "secretaria seguridad": r"(?=.*secretaria)(?=.*seguridad)",
        "secretaria ambiente": r"(?=.*secretaria)(?=.*ambiente)",
        "secretaria desarrollo economico": r"(?=.*secretaria)(?=.*economico)",
        "secretaria desarrollo social": r"(?=.*secretaria)(?=.*desarrollo)(?=.*social)",
        "secretaria gobierno": r"(?=.*secretaria)(?=.*gobierno)",
        "secretaria integracion social": r"(?=.*secretaria)(?=.*integracion)",
        "secretaria movilidad": r"(?=.*secretaria)(?=.*movilidad)",
        "secretaria mujer": r"(?=.*secretaria)(?=.*mujer)",
        "secretaria planeacion": r"(?=.*secretaria)(?=.*planeacion)",
        "secretaria salud": r"(?=.*secretaria)(?=.*salud)",
        "secretaria seguridad": r"(?=.*secretaria)(?=.*seguridad)",
        "secretaria habitat": r"(?=.*secretaria)(?=.*habitat)",
        "secretaria general": r"(?=.*secretaria)(?=.*general)",
        "secretaria juridica distrital": r"(?=.*secretaria)(?=.*juridica)",
        "secretaria transito": r"(?=.*secretaria)(?=.*transito)", 
        
        #OTROS
        "adres": r"\badministradora\b|(?=.*recursos)(?=.*salud)|(?=.*administracion)(?=.*recursos)(?=.*salud)",
        "acueducto": r"acueducto",
        "fiscalia": r"fiscalia",
        "contraloria": r"contraloria",
        "procuraduria": r"procuraduria",
        "registraduria nacional" : r"registraduria", 
        "migracion colombia": r"(?=.*migra[cs]?ion)(?=.*colombia)|(?=.*unidad)(?=.*migra[cs]?ion)", 
        "dian": r"\bdian\b|(?=.*direccion)(?=.*impuestos)|(?=.*direccion)(?=.*aduanas)",
        "comision nacional servicio civil": r"cnsc|(?=.*comi[sc]?ion)(?=.*servicio)(?=.*civil)|(?=.*servicio)(?=.*civil)(?=.*nacional)"
            }

### Fuzzy Matcher

In [7]:
nlp = spacy.blank("es")

In [8]:
doc_matches = []
matcher = FuzzyMatcher(nlp.vocab) # Creación de un objeto FuzzyMatcher 

# Agregar patrones al matcher junto con sus identificadores
#UARIV
matcher.add("uariv", [nlp("uariv"), nlp("unidad atencion reparacion integral victimas"), nlp("unidad victimas")])
#pensiones 
matcher.add("colpensiones", [nlp("colpensiones")])
matcher.add("colfondos", [nlp("colfondos")])
matcher.add("skandia afp", [nlp("skandia")])
matcher.add("proteccion afp", [nlp("proteccion afp"), nlp("afp proteccion")])
matcher.add("porvenir afp", [nlp("porvenir afp"), nlp("afp porvenir")])
#eps
matcher.add("asmet salud eps", [nlp("asmet salud"), nlp("asmet"), nlp("eps asmet salud"), nlp("asmet salud eps")])
matcher.add("asociacion indigena cauca aic eps", [nlp("aic epsi"), nlp("epsi aic"), nlp("aic eps"), nlp("eps aic"), nlp("asociacion indigena cauca"), nlp("asociacion indigena cauca epsi"), nlp("epsi asociacion indigena cauca"),nlp("asociacion indigena cauca eps"), nlp("eps asociacion indigena cauca")])
matcher.add("aliansalud eps", [nlp("aliansalud"), nlp("aliansalud eps"), nlp("eps aliansalud")])
matcher.add("ambuq eps", [nlp("asociacion mutual barrios unidos quibdo"), nlp("ambuq")])
matcher.add("anas wayuu eps", [nlp("anas wayuu")])
matcher.add("cajacopi eps", [nlp("cajacopi eps"), nlp("eps cajacopi")])
matcher.add("capital salud eps", [nlp("capital salud")])
matcher.add("capresoca eps", [nlp("capresoca")])
matcher.add("coomeva eps", [nlp("coomeva eps"), nlp("eps coomeva")])
matcher.add("coosalud eps", [nlp("coosalud")])
matcher.add("comparta eps", [nlp("comparta eps"), nlp("eps comparta")])
matcher.add("compensar eps", [nlp("compensar eps"), nlp("eps compensar")])
matcher.add("comfachoco eps", [nlp("comfachoco eps"), nlp("eps comfachoco")])
matcher.add("comfaca eps", [nlp("comfaca eps"), nlp("eps comfaca")])
matcher.add("comfacor eps", [nlp("comfacor eps"), nlp("eps comfacor")])
matcher.add("comfasucre eps", [nlp("comfasucre eps"), nlp("eps comfasucre")])
matcher.add("comfanorte eps", [nlp("comfanorte eps"), nlp("eps comfanorte")])
matcher.add("comfaoriente eps", [nlp("comfaoriente eps"), nlp("eps comfaoriente")])
matcher.add("comfamiliar eps", [nlp("comfamiliar eps"), nlp("eps comfamiliar")])
matcher.add("comfenalco eps", [nlp("comfenalco eps"), nlp("eps comfenalco")])
matcher.add("comfandi eps", [nlp("comfandi eps"), nlp("eps comfandi")])
matcher.add("convida eps", [nlp("convida eps"), nlp("eps convida")])
matcher.add("cosmitet eps", [nlp("cosmitet eps"), nlp("eps cosmitet")])
matcher.add("dusakawi eps", [nlp("dusakawi")])
matcher.add("ecoopsos eps", [nlp("ecoopsos")])
matcher.add("emssanar eps", [nlp("emssanar")])
matcher.add("famisanar eps", [nlp("famisanar")])
matcher.add("familiar de colombia eps", [nlp("familiar colombia eps"), nlp("eps familiar colombia")])
matcher.add("mallamas eps", [nlp("mallamas eps"), nlp("eps mallamas")])
matcher.add("medimas eps", [nlp("medimas eps"), nlp("eps medimas")])
matcher.add("mutualser eps", [nlp("mutualser eps"), nlp("eps mutualser")])
matcher.add("nueva eps", [nlp("nueva eps")])
matcher.add("pijaos salud eps", [nlp("pijaos salud")])
matcher.add("saludcoop eps", [nlp("saludcoop"), nlp("saludcoop eps"), nlp("eps saludcoop")])
matcher.add("salud bolivar eps", [nlp("salud bolivar eps"), nlp("eps salud bolivar")])
matcher.add("salud mia eps", [nlp("salud mia eps"), nlp("eps salud mia")])
matcher.add("salud total eps", [nlp("salud total")])
matcher.add("sanitas eps", [nlp("sanitas eps"), nlp("colsanitas eps"), nlp("eps sanitas"), nlp("eps colsanitas")])
matcher.add("savia salud eps", [nlp("savia salud"), nlp("savia salud eps"), nlp("eps savia salud"), nlp("alianza medellin antioquia eps"), nlp("eps alianza medellin antioquia")])
matcher.add("servicio occidental de salud eps", [nlp("sos eps"), nlp("eps sos"), nlp("servicio occidental salud")])
matcher.add("sura eps", [nlp("sura eps"), nlp("eps sura")])

#ministerios
matcher.add("ministerio del interior", [nlp("ministerio interior"), nlp("mininterior")])
matcher.add("ministerio de relaciones exteriores", [nlp("ministerio de relaciones exteriores"), nlp("cancilleria")])
matcher.add("ministerio de hacienda y credito publico", [nlp("ministerio de hacienda"), nlp("minhacienda")])
matcher.add("ministerio de justicia y del derecho", [nlp("ministerio de justicia y del derecho"),  nlp("minjusticia")])
matcher.add("ministerio de defensa nacional", [nlp("ministerio de defensa"),  nlp("mindefensa")])
matcher.add("ministerio de agricultura y desarrollo rural", [nlp("ministerio de agricultura"),  nlp("minagricultura")])
matcher.add("ministerio de salud y proteccion social", [nlp("ministerio de salud"),  nlp("minsalud")])
matcher.add("ministerio de trabajo", [nlp("ministerio de trabajo"),  nlp("mintrabajo")])
matcher.add("ministerio de minas y energia", [nlp("ministerio de minas y energia"),  nlp("minminas")])
matcher.add("ministerio de comercio industria y turismo", [nlp("ministerio comercio industria"),  nlp("mincomercio")])
matcher.add("ministerio de educacion nacional", [nlp("ministerio educacion"),  nlp("mineducacion")])
matcher.add("ministerio de ambiente y desarrollo sostenible", [nlp("ministerio ambiente"),  nlp("minambiente")])
matcher.add("ministerio de vivienda ciudad y territorio", [nlp("ministerio vivienda"),  nlp("minvivienda")])
matcher.add("ministerio de tecnologias de la informacion y las comunicaciones", [nlp("ministerio tecnologias informacion comunicaciones"), nlp("ministerio tic"), nlp("mintic")])
matcher.add("ministerio de transporte", [nlp("ministerio transporte"),  nlp("mintransporte")])
matcher.add("ministerio de cultura", [nlp("ministerio cultura"),  nlp("mincultura")])
matcher.add("ministerio de ciencia tecnologia e innovacion", [nlp("ministerio ciencia tecnologia innovacion"), nlp("minciencias")])
matcher.add("ministerio de deporte", [nlp("ministerio de deporte"),  nlp("mindeporte")])
matcher.add("ministerio de igualdad", [nlp("ministerio igualdad"),  nlp("minequidad")])



# Ratios de las entidades 

In [9]:
#Estos ratios o  coeficientes de similitud se pueden editar a conveniencia para luego usarlos en la clasificacion
clasificaciones = {
    ("uariv", 83): "uariv",
    #pensiones
    ("colpensiones", 86): "colpensiones",
    ("colfondos", 92): "colfondos",
    ("skandia afp", 92): "skandia afp",
    ("proteccion afp", 96): "proteccion afp",
    ("porvenir afp", 87): "porvenir afp",
    #eps
    ("asmet salud eps", 86): "asmet salud eps",
    ("asociacion indigena cauca aic eps", 93): "asociacion indigena cauca aic eps",
    ("aliansalud eps", 91): "aliansalud eps",
    ("ambuq eps", 86): "ambuq eps",
    ("anas wayuu eps", 75): "anas wayuu eps",
    ("cajacopi eps", 75): "cajacopi eps",
    ("capital salud eps", 92): "capital salud eps",
    ("capresoca eps", 78): "capresoca eps",
    ("coomeva eps", 86): "coomeva eps",
    ("coosalud eps", 84): "coosalud eps",
    ("comparta eps", 80): "comparta eps",
    ("compensar eps", 92): "compensar eps",
    ("comfachoco eps", 83): "comfachoco eps",
    ("comfaca eps", 83): "comfaca eps",
    ("comfacor eps", 81): "comfacor eps",
    ("comfasucre eps", 84): "comfasucre eps",
    ("comfanorte eps", 84): "comfanorte eps",
    ("comfaoriente eps", 94): "comfaoriente eps",
    ("comfamiliar eps", 87): "comfamiliar eps",
    ("comfenalco eps", 93): "comfenalco eps",
    ("comfandi eps", 92): "comfandi eps",
    ("convida eps", 91): "convida eps",
    ("cosmitet eps", 80): "cosmitet eps",
    ("dusakawi eps", 75): "dusakawi eps",
    ("ecoopsos eps", 77): "ecoopsos eps",
    ("emssanar eps", 84): "emssanar eps",
    ("famisanar eps", 86): "famisanar eps",
    ("familiar de colombia eps", 86): "familiar colombia eps",
    ("mallamas eps", 81): "mallamas eps",
    ("medimas eps", 91): "medimas eps",
    ("mutualser eps", 78): "mutualser eps",
    ("nueva eps", 89): "nueva eps",
    ("pijaos salud eps", 85): "pijaos salud eps",
    ("saludcoop", 76): "saludcoop",
    ("salud bolivar eps", 90): "salud bolivar eps",
    ("salud mia eps", 94): "salud mia eps",
    ("salud total eps", 87): "salud total eps",
    ("sanitas eps", 91): "sanitas eps",
    ("savia salud eps", 89): "savia salud eps",
    ("servicio occidental de salud eps", 89): "servicio occidental de salud eps",
    ("sura eps", 86): "sura eps",
    
    #ministerios
    
    ("ministerio del interior", 95): "ministerio del interior",
    ("ministerio de relaciones exteriores", 95): "ministerio de relaciones exteriores",
    ("ministerio de hacienda y credito publico", 93): "ministerio de hacienda y credito publico",
    ("ministerio de justicia y del derecho", 85): "ministerio de justicia y del derecho",
    ("ministerio de defensa nacional", 84): "ministerio de defensa nacional",
    ("ministerio de agricultura y desarrollo rural", 89): "ministerio de agricultura y desarrollo rural",
    ("ministerio de salud y proteccion social", 89): "ministerio de salud y proteccion social",
    ("ministerio de trabajo", 92): "ministerio de trabajo",
    ("ministerio de minas y energia", 97): "ministerio de minas y energia",
    ("ministerio de comercio industria y turismo", 96): "ministerio de comercio industria y turismo",
    ("ministerio de educacion nacional", 87): "ministerio de educacion nacional",
    ("ministerio de ambiente y desarrollo sostenible", 86): "ministerio de ambiente y desarrollo sostenible",
    ("ministerio de vivienda ciudad y territorio", 95): "ministerio de vivienda ciudad y territorio",
    ("ministerio de tecnologias de la informacion y las comunicaciones", 99): "ministerio de tecnologias de la informacion y las comunicaciones",
    ("ministerio de transporte", 95): "ministerio de transporte",
    ("ministerio de cultura", 99): "ministerio de cultura",
    ("ministerio de ciencia tecnologia e innovacion", 98): "ministerio de ciencia tecnologia e innovacion",
    ("ministerio de deporte", 92): "ministerio de deporte",
    ("ministerio de igualdad", 99): "ministerio de igualdad"

    
}

# Funcion clasificacion y fuzzy matching 

In [10]:
#La funcion solo clasifica a partir de las regex definidas para las entidades 
def clasificacion_entidades(texto_limpio, departamentos):
    entidades_encontradas = []
    for entidad, patron in entidades.items():
        if re.findall(patron, texto_limpio):
            entidades_encontradas.append(entidad)
            if entidad == "gobernacion": #en caso de encontrar "gobernacion", se itera en la lista de departamentos para 
                                         #identificar la gobernacion de que departamento es
                
                for departamento in departamentos:
                    if re.search(r'{}'.format(departamento), texto_limpio):
                        return entidad + " " + departamento
    return ', '.join(entidades_encontradas) if entidades_encontradas else "otro"

#Fuzzy matching
mach_values = [] # Lista para almacenar los mejores identificadores de coincidencia encontrados
ratio_values = [] # Lista para almacenar los mejores coeficientes de similitud encontrados

# Iterar a través de los textos en accionados["texto_limpio"]
for text in accionados["texto_limpio"]:
    doc = nlp(text)
    matches = matcher(doc)
    best_match_id = ""  # Variable para almacenar el mejor identificador de coincidencia encontrado hasta ahora
    best_ratio = 0 # Variable para almacenar el mejor coeficiente de similitud encontrado hasta ahora

    # Iterar a través de las coincidencias encontradas en el texto
    for match_id, start, end, ratio, pattern in matches:
        if ratio > best_ratio:  # Verifica si el ratio de la coincidencia es mayor que el mejor ratio encontrado
            best_match_id = match_id
            best_ratio = ratio

    mach_values.append(best_match_id)
    ratio_values.append(best_ratio)

# valores de "mach" y ratio
accionados["clasificacion"] = mach_values

# Clasificación basada en los ratios

for i in range(len(ratio_values)):
    ratio = ratio_values[i]
    classifications = []

    for key, value in clasificaciones.items():
        if mach_values[i] == key[0] and ratio > key[1]:
            classifications.append(value)

    if len(classifications) > 0:
        accionados.loc[i, "clasificacion"] = ", ".join(classifications)
    else:
         # Si no se encontraron clasificaciones, se llama a la función clasificacion_entidades
        # para asignar la clasificación de clasificacion_entidades de las regex
        accionados.loc[i, "clasificacion"] = clasificacion_entidades(accionados.loc[i, "texto_limpio"], departamentos)

In [11]:
accionados["ratio"] = ratio_values

In [12]:
#Eliminar columnas de texto limpio, otros y ratio para dejar solo la columna de clasificación  
columnas_eliminar = ['texto_limpio', 'otros', 'ratio']
# Eliminar las columnas especificadas 
accionados = accionados.drop(columnas_eliminar, axis=1)

In [13]:
accionados.to_excel('accionados_verificacion.xlsx', index=False)

# Matriz de confusion y reporte

In [14]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import io

#se toma la columna etiquetada y la que clasificó el codigo y 
#Se eliminan las filas que no estan etiquetadas para hacer la matriz
accionados.dropna(subset=["clasificacion",  "Etiqueta"], inplace=True)
y_true = accionados.Etiqueta
y_pred = accionados.clasificacion


In [15]:
# Generar informe de clasificación
report = classification_report(y_true, y_pred, output_dict=True, zero_division=1)
# Crear DataFrame a partir del informe
reporte = pd.DataFrame(report).transpose()
# Restablecer el índice del DataFrame
reporte.reset_index(inplace=True)
# Cambiar el nombre de la columna 'index' a 'entidad'
reporte.rename(columns={'index': 'entidad'}, inplace=True)
reporte

,entidad,precision,recall,f1-score,support
0,aliansalud eps,0.950000,0.987013,0.968153,154.000000
1,ambuq eps,0.909091,0.833333,0.869565,12.000000
2,anas wayuu eps,1.000000,1.000000,1.000000,14.000000
3,asmet salud eps,1.000000,0.998958,0.999479,1919.000000
4,asociacion indigena cauca aic eps,0.000000,1.000000,0.000000,0.000000
5,asociacion indigena del cauca aic eps,1.000000,0.000000,0.000000,209.000000
6,cajacopi eps,1.000000,1.000000,1.000000,1390.000000
7,capital salud eps,1.000000,1.000000,1.000000,1128.000000
8,capresoca eps,1.000000,0.985876,0.992888,354.000000
9,colfondos,0.000000,1.000000,0.000000,0.000000
